In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
district_df = pd.read_csv("Section data/house party vote share by district 1948-2020.csv")

In [3]:
train_data = district_df[["State", "district_num", "raceYear", "CD", "RepStatus", "DemStatus", "RepVotesMajorPercent"]]

In [4]:
train_data["Rep Inc"] = train_data["RepStatus"] == "Incumbent"
train_data["Dem Inc"] = train_data["DemStatus"] == "Incumbent"

<ipython-input-4-d3516bb75369>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["Rep Inc"] = train_data["RepStatus"] == "Incumbent"
<ipython-input-4-d3516bb75369>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["Dem Inc"] = train_data["DemStatus"] == "Incumbent"


In [5]:
contested_races = train_data[~((train_data["RepVotesMajorPercent"] >= 99) | (train_data["RepVotesMajorPercent"] <= 1))]

In [6]:
incumb_df = pd.read_csv("Section data/incumb_dist_1948-2022 (2).csv")

In [7]:
ratings_df = pd.read_csv('Section data/expert_rating.csv')

In [8]:
ratings_no_al = ratings_df[ratings_df["district"].str.isnumeric()]
ratings_no_al["district"] = ratings_no_al["district"].astype(int)

<ipython-input-8-22bd3403cf04>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings_no_al["district"] = ratings_no_al["district"].astype(int)


In [9]:
train_all = pd.merge(contested_races, ratings_no_al, how="left", left_on=["raceYear", "State", "district_num"], right_on = ["year", "state", "district"])

In [10]:
train_ratings = train_all[~train_all["year"].isna()]

In [11]:
from sklearn.linear_model import LinearRegression

In [12]:
lr = LinearRegression()
lr.fit(train_ratings[["Rep Inc","Dem Inc","avg_rating"]], train_ratings["RepVotesMajorPercent"])
lr.score(train_ratings[["Rep Inc","Dem Inc","avg_rating"]], train_ratings["RepVotesMajorPercent"])

0.7208356296975827

In [13]:
issues_df = pd.read_csv("Section data/ads_issues_2012-2018 (1).csv")

/Users/joshuadoolan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (12,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
issues_df.head()

,Unnamed: 0,creative,market,dma,issue,airdate,airtime,state,race,district,party,ad_tone,ad_purpose,est_cost,station,cycle,st_fips,cd_fips,st_cd_fips
0,1,HOUSE/AK01 YOUNG CHALLENGERS ROAR,JUNEAU,747,GUN CONTROL|HEALTHCARE|BUDGET/GOVERNMENT SPEND...,2012-08-18,17:20:35,Alaska,US HOUSE,1.0,Republican,promote,policy,20.0,KATH,2012,2.0,1.0,201
1,2,HOUSE/AK01 YOUNG CHALLENGERS ROAR,JUNEAU,747,GUN CONTROL|HEALTHCARE|BUDGET/GOVERNMENT SPEND...,2012-08-18,17:59:25,Alaska,US HOUSE,1.0,Republican,promote,policy,20.0,KATH,2012,2.0,1.0,201
2,3,HOUSE/AK01 YOUNG CHALLENGERS ROAR,JUNEAU,747,GUN CONTROL|HEALTHCARE|BUDGET/GOVERNMENT SPEND...,2012-08-18,22:23:24,Alaska,US HOUSE,1.0,Republican,promote,policy,20.0,KATH,2012,2.0,1.0,201
3,4,HOUSE/AK01 YOUNG CHALLENGERS ROAR,JUNEAU,747,GUN CONTROL|HEALTHCARE|BUDGET/GOVERNMENT SPEND...,2012-08-19,07:46:08,Alaska,US HOUSE,1.0,Republican,promote,policy,20.0,KATH,2012,2.0,1.0,201
4,5,HOUSE/AK01 YOUNG CHALLENGERS ROAR,JUNEAU,747,GUN CONTROL|HEALTHCARE|BUDGET/GOVERNMENT SPEND...,2012-08-19,19:46:44,Alaska,US HOUSE,1.0,Republican,promote,policy,30.0,KATH,2012,2.0,1.0,201


In [17]:
rep_spend_df = issues_df.groupby(["state", "district", "cycle", "party"])["est_cost"].sum().reset_index()
rep_spend_df["Rep Cost"] = rep_spend_df["est_cost"] * (rep_spend_df["party"] == "Republican")
rep_spend_df["Dem Cost"] = rep_spend_df["est_cost"] * (rep_spend_df["party"] == "Democrat")
rep_spend_df.groupby(["state", "district", "cycle"])[["Rep Cost", "Dem Cost"]].sum().reset_index()
rep_spend_df["Rep Spend Advantage"] = rep_spend_df["Rep Cost"] - rep_spend_df["Dem Cost"]
rep_spend_df["Rep Spend Proportion"] = 2*(rep_spend_df["Rep Cost"] / (rep_spend_df["Rep Cost"] + rep_spend_df["Dem Cost"]) - 0.5)
rep_spend_df

,state,district,cycle,party,est_cost,Rep Cost,Dem Cost,Rep Spend Advantage,Rep Spend Proportion
0,Alabama,1.0,2012,Republican,410050.0,410050.0,0.0,410050.0,1.0
1,Alabama,2.0,2012,Democrat,7520.0,0.0,7520.0,-7520.0,-1.0
2,Alabama,2.0,2012,Republican,177250.0,177250.0,0.0,177250.0,1.0
3,Alabama,2.0,2016,Republican,122700.0,122700.0,0.0,122700.0,1.0
4,Alabama,2.0,2018,Democrat,135250.0,0.0,135250.0,-135250.0,-1.0
...,...,...,...,...,...,...,...,...,...
1184,Wisconsin,8.0,2014,Republican,81030.0,81030.0,0.0,81030.0,1.0
1185,Wisconsin,8.0,2016,Democrat,1060080.0,0.0,1060080.0,-1060080.0,-1.0
1186,Wisconsin,8.0,2016,Republican,1389030.0,1389030.0,0.0,1389030.0,1.0
1187,Wisconsin,8.0,2018,Democrat,62580.0,0.0,62580.0,-62580.0,-1.0


In [18]:
# ad_spend = issues_df.groupby(["state", "district", "cycle"])["est_cost"].sum().reset_index()
# ad_spend["district"] = ad_spend["district"].astype(int)
# ad_spend.head()

In [19]:
train_ads = pd.merge(train_ratings, rep_spend_df, how="left", left_on=["State", "district_num", "raceYear"], right_on = ["state", "district", "cycle"])
train_ads[~train_ads["est_cost"].isna()]

,State,district_num,raceYear,CD,RepStatus,DemStatus,RepVotesMajorPercent,Rep Inc,Dem Inc,year,...,avg_rating,state_y,district_y,cycle,party,est_cost,Rep Cost,Dem Cost,Rep Spend Advantage,Rep Spend Proportion
116,Arizona,1,2018,AZ-01,Challenger,Incumbent,46.16,False,True,2018.0,...,2.583333,Arizona,1.0,2018.0,Democrat,2586880.0,0.0,2586880.0,-2586880.0,-1.0
117,Arizona,1,2018,AZ-01,Challenger,Incumbent,46.16,False,True,2018.0,...,2.583333,Arizona,1.0,2018.0,Republican,7410.0,7410.0,0.0,7410.0,1.0
118,Arizona,2,2018,AZ-02,Challenger,Challenger,45.25,False,False,2018.0,...,2.250000,Arizona,2.0,2018.0,Democrat,1967970.0,0.0,1967970.0,-1967970.0,-1.0
119,Arizona,2,2018,AZ-02,Challenger,Challenger,45.25,False,False,2018.0,...,2.250000,Arizona,2.0,2018.0,Republican,1083950.0,1083950.0,0.0,1083950.0,1.0
121,Arizona,8,2018,AZ-08,Incumbent,Challenger,55.46,True,False,2018.0,...,6.208333,Arizona,8.0,2018.0,Democrat,448890.0,0.0,448890.0,-448890.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
784,West Virginia,3,2012,WV-03,Challenger,Incumbent,46.02,False,True,2012.0,...,1.928571,West Virginia,3.0,2012.0,Republican,455780.0,455780.0,0.0,455780.0,1.0
785,Wisconsin,7,2012,WI-07,Incumbent,Challenger,56.15,True,False,2012.0,...,5.142857,Wisconsin,7.0,2012.0,Democrat,592010.0,0.0,592010.0,-592010.0,-1.0
786,Wisconsin,7,2012,WI-07,Incumbent,Challenger,56.15,True,False,2012.0,...,5.142857,Wisconsin,7.0,2012.0,Republican,1683330.0,1683330.0,0.0,1683330.0,1.0
787,Wisconsin,8,2012,WI-08,Incumbent,Challenger,56.00,True,False,2012.0,...,6.214286,Wisconsin,8.0,2012.0,Democrat,493050.0,0.0,493050.0,-493050.0,-1.0


In [20]:
train_ads[["year", "State", "district_num", "Rep Inc","Dem Inc","avg_rating", "RepVotesMajorPercent", "Rep Spend Advantage", "Rep Spend Proportion"]].to_csv("Section data/train_ads.csv", index=False)

In [9]:
district_df

,Unnamed: 0,office,year,state,st_fips,district_num,district_id,winner_party,RepVotes,RepCandidate,RepStatus,DemVotes,DemCandidate,DemStatus,RepVotesMajorPercent,DemVotesMajorPercent,winner_candidate,winner_candidate_inc,st_cd_fips
0,1,House,2020,Alabama,1,1,AL01,R,211825,"Carl, Jerry",Challenger,116949,"Averhart, James",Challenger,64.43,35.57,RepCandidate,Challenger,101
1,2,House,2020,Alabama,1,2,AL02,R,197996,"Moore, Barry",Challenger,105286,"Harvey-Hall, Phyllis",Challenger,65.28,34.72,RepCandidate,Challenger,102
2,3,House,2020,Alabama,1,3,AL03,R,217384,"Rogers, Mike D.",Incumbent,104595,"Winfrey, Adia McClellan",Challenger,67.51,32.49,RepCandidate,Incumbent,103
3,4,House,2020,Alabama,1,4,AL04,R,261553,"Aderholt, Robert",Incumbent,56237,"Neighbors, Rick",Challenger,82.30,17.70,RepCandidate,Incumbent,104
4,5,House,2020,Alabama,1,5,AL05,R,253094,"Brooks, Mo",Incumbent,0,NaN,NaN,100.00,0.00,RepCandidate,Incumbent,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16062,16063,House,1948,Wisconsin,55,7,WI07,R,64531,"Murray, Reid F.",Incumbent,37307,"Kronenwetter, Ralph E.",Challenger,63.37,36.63,RepCandidate,Incumbent,5507
16063,16064,House,1948,Wisconsin,55,8,WI08,R,70905,"Byrnes, John W.",Incumbent,53287,"Young, Martin J.",Challenger,57.09,42.91,RepCandidate,Incumbent,5508
16064,16065,House,1948,Wisconsin,55,9,WI09,R,76903,"Hull, Merlin",Incumbent,0,NaN,NaN,100.00,0.00,RepCandidate,Incumbent,5509
16065,16066,House,1948,Wisconsin,55,10,WI10,R,52124,"O'Konski, Alvin E.",Incumbent,39523,"Hoan, Daniel W.",Challenger,56.87,43.13,RepCandidate,Incumbent,5510
